In [6]:
import os
import cudf
from cuml.preprocessing import LabelEncoder

ROOT = "/rapids/notebooks/host/kaggle/kaggle"

# Load and preprocess data
# https://docs.dask.org/en/latest/remote-data-services.html
gdf_train = cudf.read_csv(os.path.join(ROOT, "train.csv"))
#gdf_train = cudf.read_csv("s3://jheaton-load-data/kaggle/train.csv", storage_options = {'anon': True})
#gdf_train = cudf.read_csv("s3://jheaton-load-data/kaggle/train.csv", storage_options = {'key': '', 'secret':''})
x = gdf_train.iloc[:,1:-9]
y = gdf_train.loc[:,'target']
y = LabelEncoder().fit_transform(y)

OSError: [Errno 22] Bad Request

In [3]:
%%time
import xgboost as xgb

# Train model
params = dict(
    objective='multi:softmax',
    max_depth=11,
    num_class=9,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.7,
    random_state=42,
    eval_metric='mlogloss',
    tree_method='gpu_hist',
    n_gpus = 1)
dtrain = xgb.DMatrix(x,y)
evallist = [(dtrain, 'train')]
clf = xgb.train(params, dtrain, 100, evallist)

[21:15:53] WARNING: /opt/conda/envs/rapids/conda-bld/xgboost_1607463112280/work/include/xgboost/generic_parameters.h:39: 
n_gpus: 
	Deprecated. Single process multi-GPU training is no longer supported.
	Please switch to distributed training with one process per GPU.
	This can be done using Dask or Spark.  See documentation for details.
[0]	train-mlogloss:2.07834
[1]	train-mlogloss:1.96656
[2]	train-mlogloss:1.86754
[3]	train-mlogloss:1.78104
[4]	train-mlogloss:1.70832
[5]	train-mlogloss:1.64178
[6]	train-mlogloss:1.57879
[7]	train-mlogloss:1.52101
[8]	train-mlogloss:1.46779
[9]	train-mlogloss:1.41734
[10]	train-mlogloss:1.37144
[11]	train-mlogloss:1.32742
[12]	train-mlogloss:1.28607
[13]	train-mlogloss:1.24687
[14]	train-mlogloss:1.20961
[15]	train-mlogloss:1.17420
[16]	train-mlogloss:1.14154
[17]	train-mlogloss:1.11053
[18]	train-mlogloss:1.08179
[19]	train-mlogloss:1.05326
[20]	train-mlogloss:1.02681
[21]	train-mlogloss:1.00142
[22]	train-mlogloss:0.97751
[23]	train-mlogloss:0.95490
